In [ ]:
# Section "Dependencies"

%%capture

import os
import json
import datetime
import requests
import getpass
import pprint

import numpy as np
import pandas as pd

!pip install zarr xarray
import xarray as xr

!pip install s3fs
import s3fs

!pip install "cdsapi>=0.7.4"
import cdsapi


In [ ]:
# Section "Define constant"

# Forcast variables
fc_variables = ['total_precipitation', '2m_temperature']

# Bouning box of interest in format [min_lon, min_lat, max_lon, max_lat]
region = [10, 45, 20, 50]

# init forecast datetime
init_time = datetime.datetime.strptime("2023-01-01", "%Y-%m-%d").replace(day=1)

# lead forecast datetime
lead_time = datetime.datetime.strptime("2023-02-01", "%Y-%m-%d").replace(day=1)

# ingested data ouput zarr file
zarr_output = 'output.zarr'


In [ ]:
# Section "Call I-Cisk cds-ingestor-process API"

# Prepare payload
icisk_api_payload = {
    "inputs": {
        "dataset": "seasonal-original-single-levels",
        "file_out": f"/tmp/{zarr_output.replace('.zarr', '')}.nc",
        "query": {
            "originating_centre": "ecmwf",
            "system": "51",
            "variable": fc_variables,
            "year": [f"{init_time.year}"],
            "month": [f"{init_time.month:02d}"],
            "day": ["01"],
            "leadtime_hour": [str(h) for h in range(24, int((lead_time - init_time).total_seconds() // 3600), 24)],
            "area": [
                region[3],
                region[0],
                region[1],
                region[2]
            ],
            "data_format": "netcdf",
        },
        "token": "YOUR-ICISK-API-TOKEN",
        "zarr_out": f"s3://saferplaces.co/test/icisk/ai-agent/{zarr_output}",
    }
}

print(); print('-------------------------------------------------------------------'); print();

pprint.pprint(icisk_api_payload)

print(); print('-------------------------------------------------------------------'); print();

icisk_api_token = 'token' # getpass.getpass("YOUR ICISK-API-TOKEN: ")

icisk_api_payload['inputs']['token'] = icisk_api_token

# Call API
root_url = 'NGROK-URL' # 'https://i-cisk.dev.52north.org/ingest'
icisk_api_response = requests.post(
    url = f'{root_url}/processes/ingestor-cds-process/execution',
    json = icisk_api_payload
)

# Display response
pprint.pprint({
    'response': icisk_api_response.json(),
    'status_code': icisk_api_response.status_code,
})


In [ ]:
# Section "Get data from I-Cisk collection"

living_lab = None
ds_variables = {
    'total_precipitation': 'tp',
    '2m_temperature': 't2m'
}
collection_name = f"seasonal-original-single-levels_{init_time.strftime('%Y%m')}_{living_lab}_{ds_variables[fc_variables[0]]}_0"

# Query collection
collection_response = requests.get(
    f'{root_url}/collections/{collection_name}/cube',
    params = {
        'bbox': ','.join(map(str, region)),
        'f': 'json'
    }
)

# Get response
if collection_response.status_code == 200:
    collection_data = collection_response.json()
else:
    print(f'Error {collection_response.status_code}: {collection_response.json()}')


In [ ]:
# Section "Build dataset"

# Parse collection output data
axes = collection_data['domain']['axes']
dims = {
    'lat': np.linspace(axes['y']['start'], axes['y']['stop'], axes['y']['num'], endpoint=True),
    'lon': np.linspace(axes['x']['start'], axes['x']['stop'], axes['x']['num'], endpoint=True),
    'time': pd.date_range(axes['time']['start'], axes['time']['stop'], axes['time']['num'])
}

params = collection_data['parameters']
ranges = collection_data['ranges']
vars = {
    name: (tuple(dims.keys()), np.array(ranges[name]['values']).reshape(ranges[name]['shape']))
    for name in params
}

# Build xarray dataset
dataset = xr.Dataset(
    data_vars = vars,
    coords = dims
)


In [ ]:
# Section "Describe dataset"

"""
Variable "dataset" is a xarray.Dataset
It has  three dimensions named:
- 'lat' : list of latitudes, 
- 'lon': list of longitudes,
- 'time': forecast timesteps
It has 51 variables named tmp_0 , tmp_1, ..., tmp_51 representing temperature forecast data of 51 different models and having shape (lat, lon, time)
"""

# Use this dataset variable to do next analysis or plots

display(dataset)
